In [1]:
from urllib import *
import json
import regex as re
import PySimpleGUI as sg
import webbrowser
import sys
import pprint
from dataclasses import dataclass
import hyperlink
import requests

In [2]:
host       = "localhost"
port       = "8983"
collection = "restaurant"
qt         = "select"
url        = 'http://' + host + ':' + port + '/solr/' + collection + '/' + qt + '?'

q          = "q=*%3A*"
#fl         = "fl=id,name"
#fq         = "fq="
#rows       = "rows=10"
wt         = "wt=json"

params     = [ q, wt ] 
p          = "&".join(params)

In [3]:
connection = request.urlopen(url+p)

if wt == "wt=json":
  response = json.load(connection) 
else:
  response = eval(connection.read())

print ("Number of hits: " + str(response['response']['numFound']))
for i in range(len(response['response']['docs'])):
    print(response['response']['docs'][i])
#pprint.pprint(response['response']['docs'][0]['webpage'])
#print(len(response['response']['docs']))

Number of hits: 1013
{'name': ['1. Positano @ RP'], 'type': ['Italian', ' European'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d21180746-Reviews-Positano_RP-Singapore.html'], 'id': '118c8d31-feaa-4986-a1f9-89c921fb25fa', '_version_': 1694013575789543424}
{'name': ['2. Grand Shanghai Restaurant'], 'type': ['Chinese', ' Asian'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1145149-Reviews-Grand_Shanghai_Restaurant-Singapore.html'], 'id': '1d0bb2f9-9dd8-4732-8736-b25bd2a63f30', '_version_': 1694013575897546752}
{'name': ['3. Fu Lin Men (NSRCC)'], 'type': ['Chinese', ' Seafood'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d17171783-Reviews-Fu_Lin_Men_NSRCC-Singapore.html'], 'id': 'ebd438d8-029f-483d-9d4b-93964d76e2fe', '_version_': 1694013575900692480}
{'name': ['4. Entre-Nous creperie'], 'type': ['French', ' European'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1193730-Reviews-En

In [5]:
layout = [[sg.Text('Search with restaurant name'),sg.InputText(), sg.Button('Search', bind_return_key=True, key="_SEARCH_")],
          [sg.Text('Search with cuisine type'),sg.InputText(), sg.Button('Search', bind_return_key=True, key="_search_")],
          [sg.Button("Quit", key = '_QUIT_')],
          [sg.Listbox([], font = ["Arial", 16],auto_size_text=True,size=(100,20),enable_events=True, key = '_listbox_')],
          [sg.Output(font = ["Arial", 16], size=(400,120), key = '_output_')]
          ]
#[sg.Output(size=(1400,650), key = '_output_')]
# Create the window http://localhost:8888/notebooks/Documents/School%20Work/2021%20Yr%203%20Sem%202/CE%204034/info-retrieval/Webpage%20demo.ipynb#
window = sg.Window("Demo Search Engine", layout, size=(1440, 850))

# Create an event loop
while True:
    event, values = window.read()
    
    # End program if user closes window or
    # presses the OK button
    if event == "_QUIT_" or event == sg.WIN_CLOSED:
        break
    if event == '_search_':
        
        wt = "wt=json"
        
        #change format of input to string
        string = str(values[0])
        
        #split terms in string into an array
        x = string.split()
        print(x)
        
        lay = []
        say = ""
        
        #transform for query
        #no need if only one term or no term
        if (len(x)>1):
            for i in range(len(x)):
                say = say + x[i]
                if (i<(len(x)-1)):
                    say = say + " OR "
            say = "("+say+")"
            
        else:
            say = x[0]
            
        print(say)
            
        
        connection = requests.get('http://localhost:8983/solr/restaurant/select', params={"q": "type"+":" + say, 'wt' : 'json', 'rows' : 100, 'indent' : True})
            #print(connection.json())
            #print("")
        response=connection.json()

        
        for i in range(len(response['response']['docs'])):
            
            sen = response['response']['docs'][i]['name']
            lay.append(sen)
            #print(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['type'])
                #p = response['response']['docs'][i]['webpage'][0]
                #lay1.append(p)
            #print(p)
            
            #print(lay1)
            #print("")
            
        #print(lay)

        window.Element('_listbox_').Update(values=lay)
        
    if event == '_SEARCH_':
        
        wt = "wt=json"
        
        #change format of input to string
        string = str(values[0])
        
        #split terms in string into an array
        x = string.split()
        print(x)
        
        lay = []
        say = ""
        
        #transform for query
        #no need if only one term or no term
        if (len(x)>1):
            for i in range(len(x)):
                say = say + x[i]
                if (i<(len(x)-1)):
                    say = say + " OR "
            say = "("+say+")"
            
        else:
            say = x[0]
            
        print(say)
            
        connection = requests.get('http://localhost:8983/solr/restaurant/select', params={"q": "name"+":" + say, 'wt' : 'json', 'rows' : 100, 'indent' : True})
        #print(connection.json())
        response=connection.json()
        
        for i in range(len(response['response']['docs'])):
        
            lay.append(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['type'])
                #p = response['response']['docs'][i]['webpage'][0]
                #lay2.append(p)
            #print(p)
            
            #print(lay2)
            #print("")

        window.Element('_listbox_').Update(values=lay)
        
    if event == '_listbox_':
        
        window['_output_'].update(value='')

        restaurant_name = values['_listbox_'][0][0]
        r = re.findall('\s(.*)',restaurant_name)
        
        connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": "restaurant"+":"+r[0], 'wt' : 'json', 'rows' : 100, 'indent' : True})
        response=connection.json()
        
        lay = []
        for i in range(len(response['response']['docs'])):
            yeet = str(response['response']['docs'][i]['comment'])
            yeet = yeet.replace("['", "")
            yeet = yeet.replace("']", "")
            yeet = yeet.replace("[\"", "")
            yeet = yeet.replace("\"]", "")
            lay.append(yeet)
            #print(response['response']['docs'][i]['comment'])
        #window.Element('_listbox_').Update(values=lay)
        i=1
        
        for comment in lay:
            print(str(i)+". "+comment)
            i+=1
        #print(r[0])
        
window.close()

* Error performing wm_overrideredirect *
